# FoodData Central (FDC) API in Python

By Sebastian Shirk and Avery Fernandez

The FoodData Central (FDC) API provides programmatic access to the FDC database, which is maintained by the U.S. Department of Agriculture (USDA). The database contains a wide range of information about food products, including nutrient content, ingredients, and serving sizes.

Please see the following resources for more information on API usage:

- Documentation
  - <a href="https://fdc.nal.usda.gov/api-guide.html" target="_blank">FoodData Central API Guide</a>
  - <a href="https://fdc.nal.usda.gov/api-spec/fdc_api.html" target="_blank">FoodData Central OpenAPI Specification</a>
  - <a href="https://app.swaggerhub.com/apis/fdcnal/food-data_central_api/1.0.1" target="_blank">FoodData Central API Swagger Documentation</a>
  - <a href="https://fdc.nal.usda.gov/api-key-signup.html" target="_blank">FoodData Central API Key Signup</a>

- Terms
  - <a href="https://www.usda.gov/privacy-policy" target="_blank">USDA Privacy Policy</a>

- Data Reuse
  - <a href="https://fdc.nal.usda.gov/" target="_blank">FoodData Central Data Reuse Information</a>

_**NOTE:**_ The FoodData Central API limits requests to a maximum of 1,000 per hour. 

*These recipe examples were tested on May 7, 2025.* 

> "U.S. Department of Agriculture, Agricultural Research Service. FoodData Central, 2019. fdc.nal.usda.gov."

> "USDA FoodData Central data are in the public domain and they are not copyrighted. They are published under <a href="https://creativecommons.org/publicdomain/zero/1.0/" target="_blank">CC0 1.0 Universal (CC0 1.0)</a>."

## Setup

### Import Libraries

The following external libraries need to be installed into your environment to run the code examples in this tutorial:
* <a href="https://github.com/psf/requests" target="_blank">requests</a>
* <a href="https://github.com/theskumar/python-dotenv" target="_blank">python-dotenv</a>
* <a href="https://github.com/ipython/ipykernel" target="_blank">ipykernel</a>
* <a href="https://github.com/pola-rs/polars" target="_blank">polars</a>

We import the libraries used in this tutorial below:

In [3]:
import requests
import polars as pl
from pprint import pprint
import os
from dotenv import load_dotenv

### Import API Key

An API key is required to access the FoodData Central API. You can sign up for one at the <a href="https://fdc.nal.usda.gov/api-key-signup.html" target="_blank">FoodData Central API Key Signup</a> page. 

We keep our API key in a separate file, a `.env` file, and use the `dotenv` library to access it. If you use this method, create a file named `.env` in the same directory as this notebook and add the following line to it:

```text
FDC_API_KEY=PUT_YOUR_API_KEY_HERE
```

In [4]:
load_dotenv()
try:
    API_KEY = os.environ["FDC_API_KEY"]
except KeyError:
    print("API key not found. Please set 'FDC_API_KEY' in your .env file.")
else:
    print("Environment and API key successfully loaded.")

Environment and API key successfully loaded.


## 1. Searching for Different Brands of Food and Obtaining Their Data

In [ ]:
BASE_URL = 'https://api.nal.usda.gov/fdc/v1/'

endpoint = 'foods/search'
# Change the query to search for different foods and the pageSize to get more or fewer results
params = {
    "query": "cheese",
    "dataType": "Branded",
    "pageSize": 2,
    "pageNumber": 0,
    "sortBy": "dataType.keyword",
    "sortOrder": "asc",
    "brandOwner": "",
    "api_key": API_KEY,
}

# Used to get all the foods of that category in the database
try:
    response = requests.get(f"{BASE_URL}{endpoint}", params=params)
    # Raise an error for bad responses
    response.raise_for_status()
    data = response.json()
    results = data.get('foods', [])
    # Print the first result in a readable format
    pprint(results[0], depth=1)
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    results = []

{'allHighlightFields': '',
 'brandName': 'FERNDALE FARMSTEAD',
 'brandOwner': 'Ferndale Farmstead LLC',
 'dataSource': 'LI',
 'dataType': 'Branded',
 'description': 'CHEESE',
 'fdcId': 1943515,
 'finalFoodInputFoods': [],
 'foodAttributeTypes': [],
 'foodAttributes': [],
 'foodCategory': 'Cheese',
 'foodMeasures': [],
 'foodNutrients': [...],
 'foodVersionIds': [],
 'gtinUpc': '853910006170',
 'householdServingFullText': '1 ONZ',
 'ingredients': 'MILK, SALT, CULTURES, ENZYMES.',
 'marketCountry': 'United States',
 'microbes': [],
 'modifiedDate': '2018-08-11',
 'packageWeight': '9.5 oz/269 g',
 'publishedDate': '2021-07-29',
 'score': 662.3066,
 'servingSize': 28.399999618530273,
 'servingSizeUnit': 'g',
 'tradeChannels': [...]}


In [6]:
# Used to look up each individual food to get more details
if len(results) > 0:
    # Get the FDC ID of the first food item
    fdc_id = results[0]['fdcId']
    endpoint = f"food/{fdc_id}"
    params = {
        "api_key": API_KEY,
    }

    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        # Raise an error for bad responses
        response.raise_for_status()
        # Print the food details in a readable format
        pprint(response.json(), depth=1)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching food details: {e}")

{'availableDate': '8/11/2018',
 'brandName': 'FERNDALE FARMSTEAD',
 'brandOwner': 'Ferndale Farmstead LLC',
 'brandedFoodCategory': 'Cheese',
 'dataSource': 'LI',
 'dataType': 'Branded',
 'description': 'CHEESE',
 'discontinuedDate': '',
 'fdcId': 1943515,
 'foodAttributes': [...],
 'foodClass': 'Branded',
 'foodComponents': [],
 'foodNutrients': [...],
 'foodPortions': [],
 'foodUpdateLog': [...],
 'gtinUpc': '853910006170',
 'householdServingFullText': '1 ONZ',
 'ingredients': 'MILK, SALT, CULTURES, ENZYMES.',
 'labelNutrients': {},
 'marketCountry': 'United States',
 'modifiedDate': '8/11/2018',
 'packageWeight': '9.5 oz/269 g',
 'publicationDate': '7/29/2021',
 'servingSizeUnit': 'g'}


## 2. Creating a Nutrient Table

The function below creates a table of nutrients for a given food item. 

In [7]:
food_data = []
nutrient_data = []

# Loop through the food items and extract the relevant data
for food in results:
    fdc_id = food['fdcId']
    
    endpoint = f"food/{fdc_id}"
    params = {
        "api_key": API_KEY,
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        # Raise an error for bad responses
        response.raise_for_status()
        food_details = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching food details: {e}")
        continue

    # Extract the relevant data from the food item
    food_data.append([
        fdc_id,
        food_details.get("brandName", ""),
        food_details.get("brandOwner", ""),
        food_details.get("marketCountry", ""),
        food_details.get("description", ""),
        food_details.get("brandedFoodCategory", ""),
        ""
    ])

    # Extract the nutrient data
    serving_size = food_details.get("servingSize", "")
    serving_size_unit = food_details.get("servingSizeUnit", "")
    nutrient_data.append([
        fdc_id,
        "Serving Size",
        f"{serving_size} {serving_size_unit}"
    ])

    # Loop through the food's nutrients and extract the relevant data
    for nutrient in food_details.get("foodNutrients", []):
        nutrient_serving_size = nutrient.get("amount", "")
        nutrient_serving_size_unit = nutrient.get("unitName", "")
        nutrient_data.append([
            fdc_id,
            nutrient.get("nutrient", {}).get("name", ""),
            f"{nutrient_serving_size} {nutrient_serving_size_unit}",
        ])

In [8]:
food_df = pl.DataFrame(
    food_data,
    schema=["FDCID", "Brand Name", "Brand Owner", "Market Country", "Description", 
            "Food Category", "Calculated from value per serving size measure"],
    orient="row"
)
food_df

FDCID,Brand Name,Brand Owner,Market Country,Description,Food Category,Calculated from value per serving size measure
i64,str,str,str,str,str,str
1943515,"""FERNDALE FARMSTEAD""","""Ferndale Farmstead LLC""","""United States""","""CHEESE""","""Cheese""",""""""
2083541,"""S.J. FALBO""","""American Pride Food Corp.""","""United States""","""CHEESE""","""Cheese""",""""""


In [9]:
nutrient_df = pl.DataFrame(
    nutrient_data,
    schema=["FDCID", "Nutrient Name", "Nutrient Amount"],
    orient="row"
)
nutrient_df

FDCID,Nutrient Name,Nutrient Amount
i64,str,str
1943515,"""Serving Size""",""" g"""
1943515,"""Vitamin A, IU""","""176.0 """
1943515,"""Fiber, total dietary""","""0.0 """
1943515,"""Total Sugars""","""0.0 """
1943515,"""Sodium, Na""","""599.0 """
…,…,…
2083541,"""Fatty acids, total trans""","""0.0 """
2083541,"""Energy""","""500.0 """
2083541,"""Cholesterol""","""100.0 """


In [10]:
nutrient_pivot_df = nutrient_df.pivot(
    index="FDCID",
    on="Nutrient Name",
    values="Nutrient Amount"
)
nutrient_pivot_df

FDCID,Serving Size,"Vitamin A, IU","Fiber, total dietary",Total Sugars,"Sodium, Na",Total lipid (fat),"Carbohydrate, by difference","Fatty acids, total trans","Vitamin C, total ascorbic acid","Iron, Fe",Energy,"Fatty acids, total saturated","Calcium, Ca",Protein,Cholesterol
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1943515,""" g""","""176.0 ""","""0.0 ""","""0.0 ""","""599.0 ""","""24.65 ""","""2.46 ""","""0.0 ""","""0.0 ""","""0.0 ""","""331.0 ""","""17.61 ""","""0.0 ""","""21.13 ""","""53.0 """
2083541,"""5.0 g""","""2000.0 ""","""0.0 ""","""0.0 ""","""2000.0 ""","""40.0 ""","""40.0 ""","""0.0 ""","""0.0 ""","""0.0 ""","""500.0 ""","""30.0 ""","""1200.0 ""","""60.0 ""","""100.0 """


In [11]:
combined_df = food_df.join(nutrient_pivot_df, on="FDCID", how="left")
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_cols(8)
# Uncomment the following line to see all columns
# pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)  
print(combined_df)


shape: (2, 22)
┌─────────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬─────────┬───────────┐
│ FDCID   ┆ Brand Name ┆ Brand      ┆ Market     ┆ … ┆ Fatty     ┆ Calcium,  ┆ Protein ┆ Cholester │
│ ---     ┆ ---        ┆ Owner      ┆ Country    ┆   ┆ acids,    ┆ Ca        ┆ ---     ┆ ol        │
│ i64     ┆ str        ┆ ---        ┆ ---        ┆   ┆ total     ┆ ---       ┆ str     ┆ ---       │
│         ┆            ┆ str        ┆ str        ┆   ┆ saturated ┆ str       ┆         ┆ str       │
│         ┆            ┆            ┆            ┆   ┆ ---       ┆           ┆         ┆           │
│         ┆            ┆            ┆            ┆   ┆ str       ┆           ┆         ┆           │
╞═════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═════════╪═══════════╡
│ 1943515 ┆ FERNDALE   ┆ Ferndale   ┆ United     ┆ … ┆ 17.61     ┆ 0.0       ┆ 21.13   ┆ 53.0      │
│         ┆ FARMSTEAD  ┆ Farmstead  ┆ States     ┆   ┆           ┆          

## 3. Creating an Ingredient Table

The function below will create a table of ingredients for a given food product.

In [12]:
food_data = []

for food in results:
    fdc_id = food['fdcId']

    endpoint = f"food/{fdc_id}"
    params = {
        "api_key": API_KEY,
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        # Raise an error for bad responses
        response.raise_for_status()
        food_details = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching food details: {e}")
        continue

    # Extract the relevant data from the food item
    food_data.append([
        fdc_id,
        food_details.get("brandName", ""),
        food_details.get("brandOwner", ""),
        food_details.get("marketCountry", ""),
        food_details.get("description", ""),
        food_details.get("brandedFoodCategory", ""),
        food_details.get("ingredients", "")
    ])

In [13]:
food_df = pl.DataFrame(
    food_data,
    schema=["FDCID", "Brand Name", "Brand Owner", "Market Country",
             "Description", "Food Category", "Ingredients"],
    orient="row"
)

pl.Config.set_fmt_str_lengths(10000)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

print(food_df)

shape: (2, 7)
┌─────────┬────────────┬─────────────────┬─────────┬─────────────┬────────────────┬────────────────┐
│ FDCID   ┆ Brand Name ┆ Brand Owner     ┆ Market  ┆ Description ┆ Food Category  ┆ Ingredients    │
│ ---     ┆ ---        ┆ ---             ┆ Country ┆ ---         ┆ ---            ┆ ---            │
│ i64     ┆ str        ┆ str             ┆ ---     ┆ str         ┆ str            ┆ str            │
│         ┆            ┆                 ┆ str     ┆             ┆                ┆                │
╞═════════╪════════════╪═════════════════╪═════════╪═════════════╪════════════════╪════════════════╡
│ 1943515 ┆ FERNDALE   ┆ Ferndale        ┆ United  ┆ CHEESE      ┆ Cheese         ┆ MILK, SALT,    │
│         ┆ FARMSTEAD  ┆ Farmstead LLC   ┆ States  ┆             ┆                ┆ CULTURES,      │
│         ┆            ┆                 ┆         ┆             ┆                ┆ ENZYMES.       │
│ 2083541 ┆ S.J. FALBO ┆ American Pride  ┆ United  ┆ CHEESE      ┆ Cheese    

## 4. Specific Food Item Lookup

These functions will allow us to retrieve information about specific food items from either of the tables we created earlier via the FDCID.

Note that these functions rely on at least one of the tables to be created in the previous steps.

In [14]:
# Retrieve the FDCID of the first food item in the results
if len(results) > 0:
    # Index of the food item to retrieve
    idx = 0  
    food = results[idx]
    fdc_id = food.get('fdcId')
    print(f"FDCID of idx {idx}: {fdc_id}")

    # Fetch and display detailed information about the food item
    if fdc_id:
        try:
            response = requests.get(f'{BASE_URL}food/{fdc_id}', params={'api_key': API_KEY})
            response.raise_for_status()
            pprint(response.json())
        except requests.exceptions.RequestException as e:
            print(f"Error fetching food details: {e}")
else:
    print("No results available to retrieve FDCID.")

FDCID of idx 0: 1943515
{'availableDate': '8/11/2018',
 'brandName': 'FERNDALE FARMSTEAD',
 'brandOwner': 'Ferndale Farmstead LLC',
 'brandedFoodCategory': 'Cheese',
 'dataSource': 'LI',
 'dataType': 'Branded',
 'description': 'CHEESE',
 'discontinuedDate': '',
 'fdcId': 1943515,
 'foodAttributes': [{'id': 2090445,
                     'name': 'Added Package Weight',
                     'value': 9}],
 'foodClass': 'Branded',
 'foodComponents': [],
 'foodNutrients': [{'amount': 176.0,
                    'foodNutrientDerivation': {'code': 'LCCD',
                                               'description': 'Calculated from '
                                                              'a daily value '
                                                              'percentage per '
                                                              'serving size '
                                                              'measure',
                                               'id': 75}